# Covid-19

In [2]:
import pandas as pd 
import datetime as datetime 
from datetime  import timedelta
from dateutil.relativedelta import relativedelta
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
sns.set()
print('Setup completed ^_____^')

Setup completed ^_____^


## Case.csv
한국의 COVID-19 감염 사례 데이터

In [124]:
# Case.csv 데이터 로딩

case_df = pd.read_csv("./data/covid19/Case.csv")
display(case_df.head())
display(case_df.describe())
display(case_df.info())

,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Yongsan-gu,True,Itaewon Clubs,139,37.538621,126.992652
1,1000002,Seoul,Gwanak-gu,True,Richway,119,37.48208,126.901384
2,1000003,Seoul,Guro-gu,True,Guro-gu Call Center,95,37.508163,126.884387
3,1000004,Seoul,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,37.546061,126.874209
4,1000005,Seoul,Dobong-gu,True,Day Care Center,43,37.679422,127.044374


,case_id,confirmed
count,1.740000e+02,174.000000
mean,2.686216e+06,65.488506
std,1.943218e+06,355.097654
min,1.000001e+06,0.000000
25%,1.100006e+06,4.000000
50%,1.700004e+06,10.000000
75%,4.100004e+06,31.750000
max,7.000004e+06,4511.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0    case_id        174 non-null    int64 
 1   province        174 non-null    object
 2   city            174 non-null    object
 3   group           174 non-null    bool  
 4   infection_case  174 non-null    object
 5   confirmed       174 non-null    int64 
 6   latitude        174 non-null    object
 7   longitude       174 non-null    object
dtypes: bool(1), int64(2), object(5)
memory usage: 9.8+ KB


None

In [126]:
# 

tmp = case_df['infection_case'].unique()
print(tmp)
len(tmp)

['Itaewon Clubs' 'Richway' 'Guro-gu Call Center'
 'Yangcheon Table Tennis Club' 'Day Care Center' 'Manmin Central Church'
 'SMR Newly Planted Churches Group' 'Dongan Church'
 'Coupang Logistics Center' 'Wangsung Church'
 "Eunpyeong St. Mary's Hospital" 'Seongdong-gu APT'
 'Jongno Community Center' 'Samsung Medical Center'
 'Jung-gu Fashion Company' 'Yeonana News Class'
 'Korea Campus Crusade of Christ' 'Gangnam Yeoksam-dong gathering'
 'Daejeon door-to-door sales'
 'Geumcheon-gu rice milling machine manufacture' 'Shincheonji Church'
 'Guri Collective Infection' 'KB Life Insurance'
 'Yeongdeungpo Learning Institute' 'Gangnam Dongin Church'
 'Biblical Language study meeting' 'Seocho Family'
 'Anyang Gunpo Pastors Group' 'Samsung Fire & Marine Insurance'
 'SJ Investment Call Center' 'Yongin Brothers'
 'Seoul City Hall Station safety worker' 'Uiwang Logistics Center'
 'Orange Life' 'Daezayeon Korea' 'overseas inflow' 'contact with patient'
 'etc' 'Onchun Church' 'Suyeong-gu Kindergarten'
 

81

## PatientInfo

In [128]:
PatientInfo_df = pd.read_csv("./data/covid19/PatientInfo.csv")
display(PatientInfo_df.head())
display(PatientInfo_df.describe())
display(PatientInfo_df.info())

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,male,50s,Korea,Seoul,Gangseo-gu,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,male,30s,Korea,Seoul,Jungnang-gu,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,male,50s,Korea,Seoul,Jongno-gu,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,male,20s,Korea,Seoul,Mapo-gu,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,female,20s,Korea,Seoul,Seongbuk-gu,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released


,patient_id
count,5.165000e+03
mean,2.863635e+09
std,2.074211e+09
min,1.000000e+09
25%,1.000001e+09
50%,2.000000e+09
75%,6.001000e+09
max,7.000000e+09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5165 entries, 0 to 5164
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   patient_id          5165 non-null   int64 
 1   sex                 4043 non-null   object
 2   age                 3785 non-null   object
 3   country             5165 non-null   object
 4   province            5165 non-null   object
 5   city                5071 non-null   object
 6   infection_case      4246 non-null   object
 7   infected_by         1346 non-null   object
 8   contact_number      791 non-null    object
 9   symptom_onset_date  690 non-null    object
 10  confirmed_date      5162 non-null   object
 11  released_date       1587 non-null   object
 12  deceased_date       66 non-null     object
 13  state               5165 non-null   object
dtypes: int64(1), object(13)
memory usage: 565.0+ KB


None

In [133]:
# 계산 할 수 있도록 날짜의 데이터 타입을 변경해 준다.

PatientInfo_df['confirmed_date'] = pd.to_datetime(PatientInfo_df['confirmed_date'], format = '%Y-%m-%d')

In [134]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

train_date_min = PatientInfo_df['confirmed_date'].min()
train_date_max = PatientInfo_df['confirmed_date'].max()
print('Minimum date from training set: {}'.format(train_date_min))
print('Maximum date from training set: {}'.format(train_date_max))

Minimum date from training set: 2020-01-20 00:00:00
Maximum date from training set: 2020-06-30 00:00:00


In [109]:
# PatientInfo의 null 값은 총 3개가 존재 한다. 

PatientInfo_df['confirmed_date'].isnull().sum()

3

## Policy.csv
한국의 COVID-19에 대한 정부 정책 데이터

In [135]:
Policy_df = pd.read_csv("./data/covid19/Policy.csv")
display(Policy_df.head())
display(Policy_df.describe())
display(Policy_df.info())

,policy_id,country,type,gov_policy,detail,start_date,end_date
0,1,Korea,Alert,Infectious Disease Alert Level,Level 1 (Blue),2020-01-03,2020-01-19
1,2,Korea,Alert,Infectious Disease Alert Level,Level 2 (Yellow),2020-01-20,2020-01-27
2,3,Korea,Alert,Infectious Disease Alert Level,Level 3 (Orange),2020-01-28,2020-02-22
3,4,Korea,Alert,Infectious Disease Alert Level,Level 4 (Red),2020-02-23,NaN
4,5,Korea,Immigration,Special Immigration Procedure,from China,2020-02-04,NaN


,policy_id
count,61.000000
mean,31.000000
std,17.752934
min,1.000000
25%,16.000000
50%,31.000000
75%,46.000000
max,61.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   policy_id   61 non-null     int64 
 1   country     61 non-null     object
 2   type        61 non-null     object
 3   gov_policy  61 non-null     object
 4   detail      59 non-null     object
 5   start_date  61 non-null     object
 6   end_date    24 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.5+ KB


None

In [61]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

oldest, latest = df['start_date'].min(), df['start_date'].max()
print(oldest) 
print(latest)

2020-01-03
2020-06-10


## Region.csv

In [16]:
df = pd.read_csv("./data/covid19/Region.csv")
df.head()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080


In [17]:
df.tail()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
239,61160,Gyeongsangnam-do,Haman-gun,35.272481,128.406540,16,20,0,1.19,23.74,14.7,94
240,61170,Gyeongsangnam-do,Hamyang-gun,35.520541,127.725177,13,12,0,1.01,32.65,20.9,83
241,61180,Gyeongsangnam-do,Hapcheon-gun,35.566702,128.165870,17,15,0,0.71,38.44,24.7,96
242,70000,Jeju-do,Jeju-do,33.488936,126.500423,113,123,4,1.53,15.10,6.4,1245
243,80000,Korea,Korea,37.566953,126.977977,6087,8837,340,1.56,15.67,7.2,94865


## SearchTrend.csv

In [62]:
df = pd.read_csv("./data/covid19/SearchTrend.csv")
df.head()

,date,cold,flu,pneumonia,coronavirus
0,2016-01-01,0.11663,0.05590,0.15726,0.00736
1,2016-01-02,0.13372,0.17135,0.20826,0.00890
2,2016-01-03,0.14917,0.22317,0.19326,0.00845
3,2016-01-04,0.17463,0.18626,0.29008,0.01145
4,2016-01-05,0.17226,0.15072,0.24562,0.01381


In [19]:
df.tail()

,date,cold,flu,pneumonia,coronavirus
1637,2020-06-25,0.12072,0.04190,0.11936,2.16993
1638,2020-06-26,0.11381,0.03027,0.11317,2.15139
1639,2020-06-27,0.10554,0.02136,0.08354,1.96848
1640,2020-06-28,0.12563,0.02872,0.08072,2.19720
1641,2020-06-29,0.14263,0.04308,0.11254,2.41501


In [63]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

oldest, latest = df['date'].min(), df['date'].max()
print(oldest) 
print(latest)

2016-01-01
2020-06-29


## SeoulFloating
국내 최대 포털사이트 네이버에서 검색된 키워드의 트렌드 데이터

In [64]:
df = pd.read_csv("./data/covid19/SeoulFloating.csv")
df.head(10)

,date,hour,birth_year,sex,province,city,fp_num
0,2020-01-01,0,20,female,Seoul,Dobong-gu,19140
1,2020-01-01,0,20,male,Seoul,Dobong-gu,19950
2,2020-01-01,0,20,female,Seoul,Dongdaemun-gu,25450
3,2020-01-01,0,20,male,Seoul,Dongdaemun-gu,27050
4,2020-01-01,0,20,female,Seoul,Dongjag-gu,28880
5,2020-01-01,0,20,male,Seoul,Dongjag-gu,30350
6,2020-01-01,0,20,female,Seoul,Eunpyeong-gu,27750
7,2020-01-01,0,20,male,Seoul,Eunpyeong-gu,27910
8,2020-01-01,0,20,female,Seoul,Gangbuk-gu,19490
9,2020-01-01,0,20,male,Seoul,Gangbuk-gu,21940


In [23]:
df.tail(10)

,date,hour,birth_year,sex,province,city,fp_num
1084790,2020-05-31,19,20,male,Seoul,Dongjag-gu,33380
1084791,2020-05-31,19,60,male,Seoul,Jungnang-gu,20360
1084792,2020-05-31,20,40,male,Seoul,Seocho-gu,38670
1084793,2020-05-31,20,70,female,Seoul,Jongno-gu,9140
1084794,2020-05-31,21,20,male,Seoul,Seodaemun-gu,25890
1084795,2020-05-31,21,40,female,Seoul,Dobong-gu,27620
1084796,2020-05-31,21,40,female,Seoul,Songpa-gu,56560
1084797,2020-05-31,21,50,female,Seoul,Gangdong-gu,38960
1084798,2020-05-31,22,60,female,Seoul,Guro-gu,25420
1084799,2020-05-31,23,40,male,Seoul,Eunpyeong-gu,38650


In [65]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

oldest, latest = df['date'].min(), df['date'].max()
print(oldest) 
print(latest)

2020-01-01
2020-05-31


## Time.csv
한국의 COVID-19 상황에 대한 시계열 데이터(테스트, 음성, 양성, 해제, 누적 사망자)

In [66]:
df = pd.read_csv("./data/covid19/Time.csv")
df.head(10)

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0
3,2020-01-23,16,22,21,1,0,0
4,2020-01-24,16,27,25,2,0,0
5,2020-01-25,16,27,25,2,0,0
6,2020-01-26,16,51,47,3,0,0
7,2020-01-27,16,61,56,4,0,0
8,2020-01-28,16,116,97,4,0,0
9,2020-01-29,16,187,155,4,0,0


In [25]:
df.tail(10)

,date,time,test,negative,confirmed,released,deceased
153,2020-06-21,0,1176463,1143971,12421,10868,280
154,2020-06-22,0,1182066,1150225,12438,10881,280
155,2020-06-23,0,1196012,1161250,12484,10908,281
156,2020-06-24,0,1208597,1175817,12535,10930,281
157,2020-06-25,0,1220478,1189015,12563,10974,282
158,2020-06-26,0,1232315,1200885,12602,11172,282
159,2020-06-27,0,1243780,1211261,12653,11317,282
160,2020-06-28,0,1251695,1219975,12715,11364,282
161,2020-06-29,0,1259954,1228698,12757,11429,282
162,2020-06-30,0,1273766,1240157,12800,11537,282


In [67]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

oldest, latest = df['date'].min(), df['date'].max()
print(oldest) 
print(latest)

2020-01-20
2020-06-30


## TimeAge.csv
한국의 연령별 코로나19 현황 시계열 데이터

In [68]:
df = pd.read_csv("./data/covid19/TimeAge.csv")
df.head(10)

,date,time,age,confirmed,deceased
0,2020-03-02,0,0s,32,0
1,2020-03-02,0,10s,169,0
2,2020-03-02,0,20s,1235,0
3,2020-03-02,0,30s,506,1
4,2020-03-02,0,40s,633,1
5,2020-03-02,0,50s,834,5
6,2020-03-02,0,60s,530,6
7,2020-03-02,0,70s,192,6
8,2020-03-02,0,80s,81,3
9,2020-03-03,0,0s,34,0


In [27]:
df.tail(10)

,date,time,age,confirmed,deceased
1079,2020-06-29,0,80s,556,139
1080,2020-06-30,0,0s,193,0
1081,2020-06-30,0,10s,708,0
1082,2020-06-30,0,20s,3362,0
1083,2020-06-30,0,30s,1496,2
1084,2020-06-30,0,40s,1681,3
1085,2020-06-30,0,50s,2286,15
1086,2020-06-30,0,60s,1668,41
1087,2020-06-30,0,70s,850,82
1088,2020-06-30,0,80s,556,139


In [69]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

oldest, latest = df['date'].min(), df['date'].max()
print(oldest) 
print(latest)

2020-03-02
2020-06-30


## TimeGender.csv
한국의 성별별 코로나19 현황 시계열 데이터

In [70]:
df = pd.read_csv("./data/covid19/TimeGender.csv")
df.head(10)

,date,time,sex,confirmed,deceased
0,2020-03-02,0,male,1591,13
1,2020-03-02,0,female,2621,9
2,2020-03-03,0,male,1810,16
3,2020-03-03,0,female,3002,12
4,2020-03-04,0,male,1996,20
5,2020-03-04,0,female,3332,12
6,2020-03-05,0,male,2149,21
7,2020-03-05,0,female,3617,14
8,2020-03-06,0,male,2345,25
9,2020-03-06,0,female,3939,17


In [29]:
df.tail(10)

,date,time,sex,confirmed,deceased
232,2020-06-26,0,male,5384,151
233,2020-06-26,0,female,7218,131
234,2020-06-27,0,male,5412,151
235,2020-06-27,0,female,7241,131
236,2020-06-28,0,male,5450,151
237,2020-06-28,0,female,7265,131
238,2020-06-29,0,male,5470,151
239,2020-06-29,0,female,7287,131
240,2020-06-30,0,male,5495,151
241,2020-06-30,0,female,7305,131


In [71]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

oldest, latest = df['date'].min(), df['date'].max()
print(oldest) 
print(latest)

2020-03-02
2020-06-30


## TimeProvince.csv
한국의 지역별 코로나19 현황 시계열 데이터

In [33]:
df = pd.read_csv("./data/covid19/TimeProvince.csv")
df.head(10)

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0
5,2020-01-20,16,Daejeon,0,0,0
6,2020-01-20,16,Ulsan,0,0,0
7,2020-01-20,16,Sejong,0,0,0
8,2020-01-20,16,Gyeonggi-do,0,0,0
9,2020-01-20,16,Gangwon-do,0,0,0


In [34]:
df.tail(10)

,date,time,province,confirmed,released,deceased
2761,2020-06-30,0,Sejong,50,47,0
2762,2020-06-30,0,Gyeonggi-do,1207,931,23
2763,2020-06-30,0,Gangwon-do,65,54,3
2764,2020-06-30,0,Chungcheongbuk-do,65,60,0
2765,2020-06-30,0,Chungcheongnam-do,167,150,0
2766,2020-06-30,0,Jeollabuk-do,27,21,0
2767,2020-06-30,0,Jeollanam-do,24,19,0
2768,2020-06-30,0,Gyeongsangbuk-do,1389,1328,54
2769,2020-06-30,0,Gyeongsangnam-do,134,128,0
2770,2020-06-30,0,Jeju-do,19,16,0


In [35]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

oldest, latest = df['date'].min(), df['date'].max()
print(oldest) 
print(latest)

2020-01-20
2020-06-30


## Weather.csv
한국 지역의 날씨 데이터

In [36]:
df = pd.read_csv("./data/covid19/Weather.csv")
df.head(10)

,code,province,date,avg_temp,min_temp,max_temp,precipitation,max_wind_speed,most_wind_direction,avg_relative_humidity
0,10000,Seoul,2016-01-01,1.2,-3.3,4.0,0.0,3.5,90.0,73.0
1,11000,Busan,2016-01-01,5.3,1.1,10.9,0.0,7.4,340.0,52.1
2,12000,Daegu,2016-01-01,1.7,-4.0,8.0,0.0,3.7,270.0,70.5
3,13000,Gwangju,2016-01-01,3.2,-1.5,8.1,0.0,2.7,230.0,73.1
4,14000,Incheon,2016-01-01,3.1,-0.4,5.7,0.0,5.3,180.0,83.9
5,15000,Daejeon,2016-01-01,1.6,-4.2,7.7,0.0,4.4,320.0,77.4
6,16000,Ulsan,2016-01-01,4.0,-1.6,12.0,0.0,2.7,320.0,53.3
7,20000,Gyeonggi-do,2016-01-01,1.6,-4.2,5.7,0.0,2.1,180.0,80.1
8,30000,Gangwon-do,2016-01-01,5.1,2.1,8.9,0.0,9.6,290.0,33.0
9,40000,Chungcheongbuk-do,2016-01-01,-1.0,-5.9,4.1,0.0,1.6,110.0,79.4


In [37]:
df.tail(10)

,code,province,date,avg_temp,min_temp,max_temp,precipitation,max_wind_speed,most_wind_direction,avg_relative_humidity
26261,16000,Ulsan,2020-06-29,21.4,19.2,24.8,65.8,5.1,110.0,93.4
26262,20000,Gyeonggi-do,2020-06-29,22.7,17.3,26.8,29.5,6.1,110.0,84.3
26263,30000,Gangwon-do,2020-06-29,22.2,19.6,25.1,2.2,3.5,90.0,85.4
26264,40000,Chunghceongbuk-do,2020-06-29,22.5,17.9,27.0,20.6,6.4,90.0,78.9
26265,41000,Chungcheongnam-do,2020-06-29,21.5,18.3,26.3,66.7,6.0,50.0,88.3
26266,50000,Jeollabuk-do,2020-06-29,22.0,19.7,26.5,27.6,4.4,90.0,85.1
26267,51000,Jeollanam-do,2020-06-29,21.9,20.0,24.0,80.5,16.8,90.0,97.9
26268,60000,Gyeongsangbuk-do,2020-06-29,20.4,17.4,24.5,43.9,4.5,160.0,90.9
26269,61000,Gyeongsangnam-do,2020-06-29,21.9,18.8,25.5,98.9,5.8,160.0,85.0
26270,70000,Jeju-do,2020-06-29,23.2,19.1,27.6,25.1,12.8,270.0,87.4


In [38]:
# 가장 최근과 예전의 시간 데이터를 확인해 보자

oldest, latest = df['date'].min(), df['date'].max()
print(oldest) 
print(latest)

2016-01-01
2020-06-29


## Explorotary Data Analysis